In [1]:
!pip3 install ivis

     |████████████████████████████████| 645kB 7.8MB/s 
  Created wheel for annoy: filename=annoy-1.16.2-cp36-cp36m-linux_x86_64.whl size=310427 sha256=8a922df768d6fee22748879099e745ed9a29bf85223eae422b1e79f56dcc7584
  Stored in directory: /root/.cache/pip/wheels/d3/d7/68/3795670ef4c6781fc10df0d6cf83b922244aa28cd9489d1176
Successfully built annoy


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from ivis import Ivis
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from keras.layers import BatchNormalization, Dense, Activation, Lambda, Input
from keras.models import Sequential, Model
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from imblearn.over_sampling import SMOTE, RandomOverSampler

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB

from statistics import median
import itertools
import time

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
base_path = './drive/My Drive/'

train_data_fname = 'xalan-2.4.csv'
test_data_fname = 'xalan-2.5.csv'

batch_size = 128
num_classes = 2

train_df = pd.read_csv(f'{base_path}/{train_data_fname}')
test_df = pd.read_csv(f'{base_path}/{test_data_fname}')

In [0]:
x_train = train_df.drop('bug', axis=1)
x_train.drop(['name', 'version', 'name.1'], axis=1, inplace=True)
y_train = train_df['bug']

x_test = test_df.drop('bug', axis=1)
x_test.drop(['name', 'version', 'name.1'], axis=1, inplace=True)
y_test = test_df['bug']

In [6]:
y_train[y_train > 1] = 1
y_test[y_test > 1] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
def parabola(x):
    return x ** 2

In [0]:
def get_model(input_units=2, units=2, n_layers=1, deep_learner=False, batchnorm=False, 
              activation='relu'):
    """
    Returns a model.
    
    Params:
    =======
    units: int - Number of input units. Only for deep learners.
    layers: int - Number of hidden layers. Only for deep learners.
    deep_learner: boolean - To use deep learner, set to True.
    batchnorm: boolean - If True, uses Batch Normalization.
    
    Returns:
    ========
    An instance of keras.Sequential or AdaBoost from sklearn.
    """
    if deep_learner:
        layers = []
        
        # Add first hidden layer set.
        first = Dense(units, input_shape=(input_units,))
        layers.append(first)
        
        if batchnorm:
            layers.append(BatchNormalization())
        
        layers.append(Activation(activation))
        
        # Add the other hidden layers.
        for i in range(n_layers - 1):
            layers.append(Dense(units))
            
            if batchnorm:
                layers.append(BatchNormalization())
            
            layers.append(Activation(activation))
        
        # Add the output layer.
        layers.append(Dense(1))
        layers.append(Activation('sigmoid'))
        
        return Sequential(layers)
    else:
        return AdaBoostClassifier()

In [0]:
def get_oversampler(name=None):
    """
    Returns an oversampler.
    
    Params:
    =======
    name: str or None - One of None, 'smote', and 'random'.
    """
    if name is None:
        return None
    if name == 'smote':
        return SMOTE()
    if name == 'random':
        return RandomOverSampler()

In [0]:
def resample(x, y):
    """
    Picks n_desired samples from x and y, trying to maintain a balanced dataset
    and oversampling when necessary.
    
    Args:
    =====
    x, y: dataset
    """
    m = len(x)
    
    # We need to pick n_desired random samples, trying to maintain a
    # balanced dataset
    c0_idx = np.where(y == 0)[0]
    c1_idx = np.where(y == 1)[0]

    n_desired = max(len(c0_idx), len(c1_idx))
        
    c0_samples = np.random.choice(c0_idx, n_desired)
    c1_samples = np.random.choice(c1_idx, n_desired)
        
    pick_idx = np.concatenate((c0_samples, c1_samples))
    return x[pick_idx], y[pick_idx]

In [0]:
def run_one_experiment(x_resampled, y_resampled, x_test, y_test, deep_learner=False, n_layers=1, batchnorm=False, lipschitz_lr=False, 
                       reduce_data=False, embedding_dims=2, k=7, oversample=False, oversampler=None,
                       activation='relu', wide=False):
    """
    Runs one experiment.
    
    Params:
    =======
    x, y: np.array - Input data
    deep_learner: boolean - If True, uses a deep learner.
    layers: int - Number of hidden layers. Deep learners only.
    batchnorm: boolean - If True, uses Batch Normalization in deep learner.
    lipschitz_lr: boolean - If True, uses LipschitzLR
    reduce_data: boolean - If True, reduces data using Ivis.
    k: int - k used in Ivis reduction.
    embedding_dims: int - Number of dimensions to reduce data to
    oversample: boolean - If True, oversamples dataset.
    oversampler: str or None - One of None, 'smote', and 'random'.
    
    Returns:
    ========
    prec, recall, f1, time - Precision, recall, and F-1 scores of the model, along with runtime
    """    
    
    # Step 3: Get the model
    if wide:
        model = get_model(input_units=x_resampled.shape[1],units=100, n_layers=n_layers, 
                      deep_learner=deep_learner, batchnorm=batchnorm,
                      activation=activation)
    else:
        model = get_model(input_units=x_resampled.shape[1],units=x_resampled.shape[1],n_layers=n_layers, 
                      deep_learner=deep_learner, batchnorm=batchnorm,
                      activation=activation)
        
    # Step 4: Train model
    if deep_learner:
        ####################
        # LipschitzLR code #
        ####################    
        def lr_schedule(epoch):
            """Learning Rate Schedule
            # Arguments
                epoch (int): The number of epochs
            # Returns
                lr (float32): learning rate
            """

            Kz = 0.
            for i in range((len(x_resampled) - 1) // batch_size + 1):
                start_i = i * batch_size
                end_i = start_i + batch_size
                xb = x_resampled[start_i:end_i]

                activ = np.linalg.norm(func([xb]))
                if activ > Kz:
                    Kz = activ

            K_ = ((num_classes - 1) * Kz) / (num_classes * batch_size)
            lr = 1 / K_
            #print('Epoch', epoch + 1, 'LR =', lr)
            return lr
        ####################
        
        if lipschitz_lr:
            model.compile(loss='binary_crossentropy',
                  optimizer='sgd')
            func = K.function([model.layers[0].input], [model.layers[-2].output])
            lr_scheduler = LearningRateScheduler(lr_schedule)
            
            start = time.time()
            model.fit(x_resampled, y_resampled, batch_size=batch_size, verbose=0, 
                      validation_data=(x_test, y_test), epochs=150, callbacks=[lr_scheduler])
            end = time.time()
        else:
            model.compile(loss='binary_crossentropy',
                  optimizer='adam')
            
            start = time.time()
            model.fit(x_resampled, y_resampled, batch_size=batch_size, verbose=0, 
                      validation_data=(x_test, y_test), epochs=100)
            end = time.time()
        
    else:
        start = time.time()
        model.fit(x_resampled, y_resampled)
        end = time.time()
    
    # Step 5: Evaluate model
    evaluators = [precision_score, recall_score, f1_score]
    if deep_learner:
        prec, recall, f1 = [f(y_test, model.predict_classes(x_test)) for f in evaluators]
    else:
        prec, recall, f1 = [f(y_test, model.predict(x_test)) for f in evaluators]
    
    return prec, recall, f1, (end - start)

In [0]:
def run_20_experiments(reduce_data=False, oversample=False, oversampler=None, 
                       embedding_dims=2, k=7, lipschitz_lr=False, activation='relu', **kwargs):
    global x_train, y_train, x_test, y_test
    prec = []
    recall = []
    f1 = []
    times = []
    
    count = 0

    # Step 1: Reduce dimensions --> (x_*_reduced, y_*)
    x_train_reduced = np.array(x_train)
    x_test_reduced = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    if reduce_data:
        ivis = Ivis(embedding_dims=embedding_dims, k=k, verbose=0)
        x_train_reduced = ivis.fit_transform(x_train_reduced)
        x_test_reduced = ivis.transform(x_test_reduced)

    # Step 2: Oversample data --> (x_resampled, y_resampled)
    x_resampled, y_resampled = x_train_reduced, y_train
    if oversample:
        #sampler = get_oversampler(name='random')
        x_resampled, y_resampled = resample(x_train_reduced, y_train)
        #x_resampled, y_resampled = sampler.fit_resample(x_train_reduced, y_train)

    while count != 20:
        try:          
            precision, rec, f1_, time_ = run_one_experiment(x_resampled, y_resampled,
                                                            x_test_reduced, y_test, 
                                                            lipschitz_lr=lipschitz_lr, 
                                                            activation=activation, **kwargs)
            prec.append(precision)
            recall.append(rec)
            f1.append(f1_)
            times.append(time_)
            count += 1
        except KeyboardInterrupt:
            raise
        except:
            print('Retrying...')
            if lipschitz_lr:
                pass
            else:
                raise
    
    print(np.percentile(f1, 25))
    print(median(f1))
    print(np.percentile(f1, 75))
    print('Precision:', prec, '\nRecall:', recall, '\nF1:', f1, '\nRuntime:', times)

In [0]:
config = {'deep_learner': True, 'wide': False, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'activation': parabola}

In [0]:
run_20_experiments(**config)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.7710507158456685
Precision: [0.7449664429530202, 0.692722371967655, 0.7039106145251397, 0.6892950391644909, 0.0, 0.7560137457044673, 0.7941176470588235, 0.0, 0.7414965986394558, 0.7873134328358209, 0.6933333333333334, 0.7655677655677655, 0.6952908587257618, 0.6940509915014165, 0.7450980392156863, 0.0, 0.711484593837535, 0.0, 0.696969696969697, 0.8525641025641025] 
Recall: [0.7900355871886121, 0.9145907473309609, 0.896797153024911, 0.9395017793594306, 0.0, 0.7829181494661922, 0.7686832740213523, 0.0, 0.7758007117437722, 0.7508896797153025, 0.9252669039145908, 0.7437722419928826, 0.8932384341637011, 0.8718861209964412, 0.8113879003558719, 0.0, 0.9039145907473309, 0.0, 0.900355871886121, 0.47330960854092524] 
F1: [0.7668393782383419, 0.7883435582822085, 0.7887323943661971, 0.7951807228915663, 0.0, 0.7692307692307692, 0.7811934900542494, 0.0, 0.7582608695652174, 0.7686703096539163, 0.7926829268292683, 0.7545126353790614, 0.7819314641744549, 0.7728706624605679, 0.776831345826235, 0.0, 0.7

In [0]:
np.percentile([0.7668393782383419, 0.7883435582822085, 0.7887323943661971, 0.7951807228915663, 0.0, 0.7692307692307692, 0.7811934900542494, 0.0, 0.7582608695652174, 0.7686703096539163, 0.7926829268292683, 0.7545126353790614, 0.7819314641744549, 0.7728706624605679, 0.776831345826235, 0.0, 0.7962382445141065, 0.0, 0.7857142857142858, 0.608695652173913], 50)

0.7710507158456685

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'activation': parabola}

In [0]:
run_20_experiments(**config)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.6914362006484838
0.7825933489650303
0.7905383273281504
Precision: [0.6625916870415648, 0.6723300970873787, 0.6675062972292192, 0.6881443298969072, 0.6650124069478908, 0.6725, 0.7018970189701897, 0.0, 0.6775, 0.6789473684210526, 0.0, 0.7204724409448819, 0.7862903225806451, 0.75, 0.0, 0.6904761904761905, 0.6532066508313539, 0.7209302325581395, 0.8133971291866029, 0.6832460732984293] 
Recall: [0.9644128113879004, 0.9857651245551602, 0.9430604982206405, 0.9501779359430605, 0.9537366548042705, 0.9572953736654805, 0.9217081850533808, 0.0, 0.9644128113879004, 0.9181494661921709, 0.0, 0.6512455516014235, 0.693950177935943, 0.5231316725978647, 0.0, 0.9288256227758007, 0.9786476868327402, 0.7722419928825622, 0.604982206405694, 0.9288256227758007] 
F1: [0.7855072463768116, 0.7994227994227995, 0.7817109144542772, 0.7982062780269058, 0.783625730994152, 0.7900146842878121, 0.796923076923077, 0.0, 0.7958883994126285, 0.7806354009077157, 0.0, 0.6841121495327103, 0.7372400756143667, 0.616352201257861

In [0]:
np.percentile([0.6625916870415648, 0.6723300970873787, 0.6675062972292192, 0.6881443298969072, 0.6650124069478908, 0.6725, 0.7018970189701897, 0.0, 0.6775, 0.6789473684210526, 0.0, 0.7204724409448819, 0.7862903225806451, 0.75, 0.0, 0.6904761904761905, 0.6532066508313539, 0.7209302325581395, 0.8133971291866029, 0.6832460732984293], 50)

0.6782236842105263

# poi 1.5 - 2.5

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}

In [0]:
run_20_experiments(**config)

0.7431350114416476
0.8244736293516781
0.8596589513752455
Precision: [0.8311688311688312, 0.8195488721804511, 0.8189655172413793, 0.8246268656716418, 0.85, 0.8524590163934426, 0.8201058201058201, 0.85, 0.8272058823529411, 0.8269230769230769, 0.8, 0.848, 0.8545454545454545, 0.7972972972972973, 0.8352059925093633, 0.8450184501845018, 0.8390804597701149, 0.8426966292134831, 0.8333333333333334, 0.8582995951417004] 
Recall: [0.7741935483870968, 0.43951612903225806, 0.7661290322580645, 0.8911290322580645, 0.5483870967741935, 0.8387096774193549, 0.625, 0.7540322580645161, 0.907258064516129, 0.6935483870967742, 0.20967741935483872, 0.8548387096774194, 0.7580645161290323, 0.47580645161290325, 0.8991935483870968, 0.9233870967741935, 0.8830645161290323, 0.907258064516129, 0.8870967741935484, 0.8548387096774194] 
F1: [0.8016701461377871, 0.5721784776902887, 0.7916666666666666, 0.8565891472868217, 0.6666666666666665, 0.8455284552845529, 0.7093821510297483, 0.7991452991452991, 0.8653846153846153, 0.7

In [0]:
np.percentile([13.897031307220459, 14.141637802124023, 14.060494422912598, 14.021902799606323, 14.289405107498169, 14.190900564193726, 14.405335426330566, 14.603696823120117, 14.536219835281372, 14.694948434829712, 14.806249618530273, 15.072721481323242, 14.984891414642334, 15.185039758682251, 15.397485494613647, 15.43086862564087, 16.006404399871826, 15.830060482025146, 15.643399238586426, 15.856942176818848], 75)

15.405831277370453

# lucene 2.2 - 2.4

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'activation': parabola}

In [0]:
run_20_experiments(**config)

0.6660946730264604
0.7409707844490452
0.751964514844476
Precision: [0.6366666666666667, 0.6483516483516484, 0.6956521739130435, 0.6485507246376812, 0.6344086021505376, 0.648936170212766, 0.6392857142857142, 0.5185185185185185, 0.6338028169014085, 0.7605633802816901, 0.6451612903225806, 0.6075949367088608, 0.6381578947368421, 0.6402640264026402, 0.6037735849056604, 0.6727941176470589, 0.6271186440677966, 0.6741573033707865, 0.6640625, 0.6653992395437263] 
Recall: [0.9408866995073891, 0.8719211822660099, 0.07881773399014778, 0.8817733990147784, 0.8719211822660099, 0.9014778325123153, 0.8817733990147784, 0.13793103448275862, 0.8866995073891626, 0.2660098522167488, 0.8866995073891626, 0.9458128078817734, 0.9556650246305419, 0.9556650246305419, 0.9458128078817734, 0.9014778325123153, 0.3645320197044335, 0.2955665024630542, 0.8374384236453202, 0.8620689655172413] 
F1: [0.7594433399602385, 0.7436974789915967, 0.1415929203539823, 0.7473903966597076, 0.7344398340248963, 0.7546391752577319, 0.74

In [0]:
np.percentile([14.766776323318481, 14.781675338745117, 14.74657654762268, 14.867595911026001, 15.08496379852295, 15.063039302825928, 15.17651629447937, 15.382344722747803, 15.378005266189575, 15.510342836380005, 15.562320709228516, 15.460089206695557, 15.664335250854492, 15.77399492263794, 15.697908639907837, 15.75919222831726, 16.056590795516968, 15.839813232421875, 15.984220027923584, 16.193334341049194], 75)

15.76289290189743

# lucene 2.0 - 2.2

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'activation': 'relu'}

In [0]:
run_20_experiments(**config)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
0.6225885225885226
0.6292968136870193
0.6450318957597402
Precision: [0.6666666666666666, 0.6595744680851063, 0.635036496350365, 0.674074074074074, 0.6615384615384615, 0.6615384615384615, 0.6589147286821705, 0.6527777777777778, 0.6541353383458647, 0.6690647482014388, 0.6610169491525424, 0.6496350364963503, 0.6692307692307692, 0.6666666666666666, 0.648, 0.6590909090909091, 0.6788321167883211, 0.6691729323308271, 0.6492537313432836, 0.65] 
Recall: [0.5694444444444444, 0.6458333333333334, 0.6041666666666666, 0.6319444444444444, 0.5972222222222222, 0.5972222222222222, 0.5902777777777778, 0.6527777777777778, 0.6041666666666666, 0.6458333333333334, 0.5416666666666666, 0.6180555555555556, 0.6041666666666666, 0.5833333333333334, 0.5625, 0.6041666666666666, 0.6458333333333334, 0.6180555555555556, 0.6041666666666666, 0.6319444444444444] 
F1: [0.6142322097378277, 0.6526315789473683, 0.6192170818505338, 0.6523297491039

# camel 1.4 - 1.6

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}

In [0]:
run_20_experiments(**config)

0.39826900076111776
0.40663768903377706
0.416744823789669
Precision: [0.3242424242424242, 0.3435114503816794, 0.37089201877934275, 0.3671497584541063, 0.36923076923076925, 0.3776824034334764, 0.3392226148409894, 0.3059490084985836, 0.348993288590604, 0.36524822695035464, 0.3755656108597285, 0.34146341463414637, 0.3489208633093525, 0.36180904522613067, 0.33221476510067116, 0.2906666666666667, 0.3620689655172414, 0.32063492063492066, 0.35443037974683544, 0.38839285714285715] 
Recall: [0.5691489361702128, 0.4787234042553192, 0.42021276595744683, 0.40425531914893614, 0.5106382978723404, 0.46808510638297873, 0.5106382978723404, 0.574468085106383, 0.5531914893617021, 0.5478723404255319, 0.44148936170212766, 0.5212765957446809, 0.5159574468085106, 0.3829787234042553, 0.526595744680851, 0.5797872340425532, 0.44680851063829785, 0.5372340425531915, 0.44680851063829785, 0.4627659574468085] 
F1: [0.41312741312741313, 0.39999999999999997, 0.3940149625935162, 0.3848101265822785, 0.42857142857142855,

# ant 1.6 - 1.7

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 2, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'activation': 'relu'}

In [0]:
run_20_experiments(**config)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where













0.5531828177130725
0.5573688914165007
0.5611978799582757
Precision: [0.46747967479674796, 0.4090909090909091, 0.4262295081967213, 0.4276729559748428, 0.43234323432343236, 0.43820224719101125, 0.42628205128205127, 0.42950819672131146, 0.4452296819787986, 0.4304635761589404, 0.4318181818181818, 0.47410358565737054, 0.4084084084084084, 0.4425087108013937, 0.4397163120567376, 0.41033434650455924, 0.46747967479674796, 0.45387453874538747, 0.42810457516339867, 0.45588235294117646] 
Recall: [0.6927710843373494, 0.8132530120481928, 0.7831325301204819, 0.8192771084337349, 0.7891566265060241, 0.7048192771084337, 0.8012048192771084, 0.7891566265060241, 0.7590361445783133, 0.7831325301204819, 0.8012048192771084, 0.7168674698795181, 0.8192771084337349, 0.7650602409638554, 0.7469879518072289, 0.8132530120481928, 0.

In [0]:
np.percentile([2.8747432231903076, 3.0046424865722656, 3.0868005752563477, 2.9611778259277344, 2.9642488956451416, 2.9796061515808105, 3.1037685871124268, 3.333313226699829, 3.3811001777648926, 3.3022077083587646, 3.4420342445373535, 3.644482374191284, 3.702660322189331, 3.6399121284484863, 3.711228132247925, 3.5213592052459717, 3.793853521347046, 3.9400522708892822, 3.9034500122070312, 3.7293715476989746],75)

3.7048022747039795

# camel 1.2 - 1.4

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}

In [0]:
run_20_experiments(**config)

0.3468843537414966
0.3556544101824185
0.3660516121917261
Precision: [0.24288840262582057, 0.2838709677419355, 0.23060796645702306, 0.2529832935560859, 0.24436090225563908, 0.2529002320185615, 0.24078624078624078, 0.25773195876288657, 0.21572212065813529, 0.2577639751552795, 0.20064205457463885, 0.23376623376623376, 0.23617021276595745, 0.24808184143222506, 0.24050632911392406, 0.22799097065462753, 0.26112759643916916, 0.2198443579766537, 0.24681933842239187, 0.24507042253521127] 
Recall: [0.7655172413793103, 0.6068965517241379, 0.7586206896551724, 0.7310344827586207, 0.4482758620689655, 0.7517241379310344, 0.6758620689655173, 0.6896551724137931, 0.8137931034482758, 0.5724137931034483, 0.8620689655172413, 0.7448275862068966, 0.7655172413793103, 0.6689655172413793, 0.6551724137931034, 0.696551724137931, 0.6068965517241379, 0.7793103448275862, 0.6689655172413793, 0.6] 
F1: [0.36877076411960136, 0.3868131868131869, 0.35369774919614144, 0.37588652482269497, 0.31630170316301703, 0.3784722222

# ivy 1.4 - 2.0

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}

In [0]:
run_20_experiments(**config)

0.23340188014101057
0.2799718621244096
0.3019969278033794
Precision: [0.22388059701492538, 0.2727272727272727, 0.25396825396825395, 0.23529411764705882, 0.22641509433962265, 0.22388059701492538, 0.28846153846153844, 0.2641509433962264, 0.24615384615384617, 0.1875, 0.25, 0.1, 0.2676056338028169, 0.1935483870967742, 0.21153846153846154, 0.2894736842105263, 0.16666666666666666, 0.24528301886792453, 0.23809523809523808, 0.26229508196721313] 
Recall: [0.375, 0.3, 0.4, 0.2, 0.3, 0.375, 0.375, 0.35, 0.4, 0.15, 0.275, 0.025, 0.475, 0.15, 0.275, 0.275, 0.05, 0.325, 0.25, 0.4] 
F1: [0.28037383177570097, 0.28571428571428564, 0.3106796116504854, 0.2162162162162162, 0.25806451612903225, 0.28037383177570097, 0.32608695652173914, 0.30107526881720426, 0.3047619047619048, 0.16666666666666663, 0.2619047619047619, 0.04000000000000001, 0.3423423423423423, 0.16901408450704225, 0.23913043478260868, 0.28205128205128205, 0.07692307692307691, 0.27956989247311825, 0.24390243902439024, 0.3168316831683169] 
Runti

# jEdit 3.2 - 4.0

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}

In [0]:
run_20_experiments(**config)

0.5067013971110585
0.5146090187468333
0.5280634386260225
Precision: [0.45217391304347826, 0.4411764705882353, 0.4339622641509434, 0.4262295081967213, 0.4224137931034483, 0.4, 0.44166666666666665, 0.4107142857142857, 0.42718446601941745, 0.4369747899159664, 0.40298507462686567, 0.3969465648854962, 0.4690265486725664, 0.4298245614035088, 0.43243243243243246, 0.4375, 0.44565217391304346, 0.4380952380952381, 0.41732283464566927, 0.4322033898305085] 
Recall: [0.6933333333333334, 0.6, 0.6133333333333333, 0.6933333333333334, 0.6533333333333333, 0.6933333333333334, 0.7066666666666667, 0.6133333333333333, 0.5866666666666667, 0.6933333333333334, 0.72, 0.6933333333333334, 0.7066666666666667, 0.6533333333333333, 0.64, 0.56, 0.5466666666666666, 0.6133333333333333, 0.7066666666666667, 0.68] 
F1: [0.5473684210526315, 0.5084745762711865, 0.5082872928176795, 0.5279187817258882, 0.513089005235602, 0.5073170731707318, 0.5435897435897435, 0.4919786096256684, 0.4943820224719101, 0.5360824742268041, 0.51674

In [0]:
np.percentile([7.785674333572388, 7.844439744949341, 7.791264295578003, 7.991171836853027, 8.059589147567749, 8.103115797042847, 8.32819128036499, 8.419585943222046, 8.507155418395996, 8.329012155532837, 8.667298793792725, 8.47134804725647, 8.769627332687378, 8.841513872146606, 8.880618333816528, 9.038430452346802, 9.054760217666626, 9.08403754234314, 9.372156381607056, 9.296295404434204], 75)

8.920071363449097

# synapse 1.0 - 1.1

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 1, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'activation': 'relu'}

In [0]:
run_20_experiments(**config)

0.4223363286264441
0.42969084787266604
0.44800000000000006
Precision: [0.40625, 0.43859649122807015, 0.43137254901960786, 0.4262295081967213, 0.3835616438356164, 0.42857142857142855, 0.4716981132075472, 0.48148148148148145, 0.4126984126984127, 0.41818181818181815, 0.38666666666666666, 0.4307692307692308, 0.45714285714285713, 0.423728813559322, 0.4027777777777778, 0.3888888888888889, 0.43333333333333335, 0.4307692307692308, 0.46153846153846156, 0.4307692307692308] 
Recall: [0.43333333333333335, 0.4166666666666667, 0.36666666666666664, 0.43333333333333335, 0.4666666666666667, 0.5, 0.4166666666666667, 0.43333333333333335, 0.43333333333333335, 0.38333333333333336, 0.48333333333333334, 0.4666666666666667, 0.5333333333333333, 0.4166666666666667, 0.48333333333333334, 0.4666666666666667, 0.43333333333333335, 0.4666666666666667, 0.4, 0.4666666666666667] 
F1: [0.41935483870967744, 0.4273504273504274, 0.39639639639639634, 0.42975206611570244, 0.42105263157894735, 0.4615384615384615, 0.44247787610

# xalan 2.4 - 2.5

In [0]:
config = {'deep_learner': True, 'wide': True, 'n_layers': 3, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'activation': 'relu'}

In [16]:
run_20_experiments(**config)

0.4125333429457141
0.4597176079734219
0.5049157512060306
Precision: [0.65, 0.648854961832061, 0.6372093023255814, 0.6909090909090909, 0.6378378378378379, 0.6511627906976745, 0.645021645021645, 0.631336405529954, 0.6487804878048781, 0.6556016597510373, 0.6244541484716157, 0.6153846153846154, 0.6589147286821705, 0.6515837104072398, 0.6871165644171779, 0.6404494382022472, 0.6497695852534562, 0.6758620689655173, 0.640625, 0.6477987421383647] 
Recall: [0.3359173126614987, 0.4392764857881137, 0.35400516795865633, 0.19638242894056848, 0.3049095607235142, 0.43410852713178294, 0.3850129198966408, 0.35400516795865633, 0.34366925064599485, 0.4082687338501292, 0.3695090439276486, 0.31007751937984496, 0.4392764857881137, 0.37209302325581395, 0.28940568475452194, 0.4418604651162791, 0.3643410852713178, 0.2532299741602067, 0.42377260981912146, 0.2661498708010336] 
F1: [0.44293015332197616, 0.5238828967642527, 0.45514950166112955, 0.3058350100603622, 0.41258741258741255, 0.5209302325581394, 0.48220064

In [0]:
# How about using Nesterov momentum? Or the one that's quickest in that GitHub repo?